In [61]:
import pandas as pd
import numpy as np
from collections import Counter
data = pd.read_csv("/Users/amy/Desktop/kc_house_data.csv")
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [2]:
date = data['date'] # Only consider year of sale date
for i in range(len(date)):
    year = date[i][0:4]
    data.iloc[i,1] = year
yr_renovated = data['yr_renovated'] # If there is a renovation, consider it as time of built
for i in range(len(yr_renovated)):
    if(yr_renovated[i] != 0):
        data.loc[i,'yr_built'] = yr_renovated[i]
data = data.dropna() # Drop all observations containing na
dummy_data = pd.get_dummies(data, columns = ['date','bedrooms','waterfront','view','condition','grade','zipcode']) # Dummy data
price = dummy_data['price'] # Price is target
newdata = dummy_data.drop(['id','yr_renovated','lat','long','price'],axis = 1) # id lat long is not relevant to our model
# Non normalized dataframe

In [3]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
d = min_max_scaler.fit_transform(newdata) #Normalized data matrix
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(d, price, test_size=0.2, random_state=42)

/Users/amy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [4]:
from flask import Flask,jsonify
import json
from flask_cors import CORS
from scipy.spatial import distance

In [43]:
def KNN(data, sqft, zipcode, year, bedroom, bathroom): #input sqft, zipcode, year, bedroom, bathroom to do prediction
    ob = pd.DataFrame(np.matrix([sqft, year, bedroom, bathroom]), columns=['sqft_living','yr_built','bedrooms','bathrooms'])
    df_zipcode = data[['sqft_living','zipcode','yr_built','bedrooms','bathrooms']] #filter by zipcode
    df_zipcode = df_zipcode[df_zipcode['zipcode'] == zipcode][['sqft_living','yr_built','bedrooms','bathrooms']]
    df_zipcode = df_zipcode.append(ob)
    df_norm = min_max_scaler.fit_transform(df_zipcode)
    dist = []
    observation = df_norm[len(df_norm)-1,:]
    for i in range(len(df_norm)-1):
        dist.append(distance.euclidean(df_norm[i,:],observation)) #find the observations with smallest distance
    index = np.argsort(dist)[:10]
    df_10 = df_zipcode.iloc[index,:]
    return df_10 #return index of dataframe

In [ ]:
def pricerange_zipcode(data,minn,maxx):
    data = data[(data.price > minn) & (data.price < maxx)]

In [72]:

data.iloc['price']

TypeError: cannot do positional indexing on <class 'pandas.core.indexes.numeric.Int64Index'> with these indexers [price] of <class 'str'>

In [20]:
def predict(clf,data,newdata,sqft,zipcode, year, bedroom, bathroom): #data:original data #newdata:dummydata
    temp = KNN(data, sqft, zipcode, year, bedroom, bathroom)
    index = temp.index
    df = newdata[newdata.index.isin(index)]
    min_max_scaler.fit(newdata) #normalize 
    df_norm = min_max_scaler.transform(df)
    top_prediction = clf.predict(df_norm) #do predition
    return np.median(top_prediction) 

In [14]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1) 
clf.fit(X_train,y_train)

/Users/amy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [74]:
app = Flask(__name__)
CORS(app)
@app.route('/project/status=<status>&sqft=<sqft>&zipcode=<zipcode>&year=<year>&bedroom=<bedroom>&bathroom=<bathroom>', methods=['GET'])
def server(status,sqft,zipcode,year,bedroom,bathroom): 
    status = int(status)
    sqft = float(sqft)
    zipcode = int(zipcode)
    year = int(year)
    bedroom = float(bedroom)
    bathroom = float(bathroom)
    if(status == 1): # st = 1 means we want to do a prediction
        price_pred = predict(clf,data,newdata,sqft,zipcode,year,bedroom,bathroom)
        return json.dumps(price_pred)
    if(status == 2): # return the dataset with columns of zipcode and price
        dict_2 = []
        for i in range(len(data)):
            dict_2.append({"lat":float(data.iloc[i,17]),"lon":data.iloc[i,18],"price":data.iloc[i,2]})
        return json.dumps(dict_2[:1000])
    if(status == 3):
        zipcode = list(Counter(data.iloc[:,16]).keys())
        count = list(Counter(data.iloc[:,16]).values())
        a_list = []
        for i in range(len(zipcode)):
            temp = data[data['zipcode'] == zipcode[i]]
            s = np.sum(temp['price'])
            average = s / count[i]
            a_list.append({'zipcode':float(zipcode[i]),'average':average})
        return json.dumps(a_list)
    
    if(status == 4): # st = 4 means we want to show the 10 similar house from prediction
        similar = KNN(data, sqft, zipcode, year, bedroom, bathroom)
        similarlist = similar.to_json()
        return json.dumps(similarlist)


In [ ]:
if __name__ == '__main__':
    app.run() #The port will be 5000

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
/Users/amy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
[2018-12-03 11:46:15,581] ERROR in app: Exception on /project/status=1&sqft=200&zipcode=98001&year=1900&bedroom=0&bathroom=0 [GET]
Traceback (most recent call last):
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask_cors/extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask/app.py", line

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


[2018-12-03 12:40:14,691] ERROR in app: Exception on /project/status=4&sqft=2000&zipcode=98125&year=1920&bedroom=2&bathroom=2&mi=2&ma=100000 [GET]
Traceback (most recent call last):
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask_cors/extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/amy/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1612, in full_di